In [1]:
from khmernlp.corpus import khmer_words

km_words = khmer_words()
print(f"Length of Khmer words {len(km_words)}")
print(km_words[:100])


Length of Khmer words 81028
['ក', 'កក', 'កកកុញ', 'កកកុះ', 'កកឈាម', 'កកទឹកកកដី', 'កករ', 'កករចិត្ត', 'កកស', 'កកស្ទះ', 'កកា', 'កកាត', 'កកាប', 'កកាយ', 'កកាយកកូរ', 'កកិច', 'កកិចកកុច', 'កកិដ', 'កកិត', 'កកិបកកុប', 'កកិល', 'កកិលចូលជិត', 'កកី', 'កកុក', 'កកុច', 'កកុធ', 'កកុធភណ្ឌ', 'កកុសន្ធៈ', 'កកូន', 'កកូរ', 'កកូរកកាយ', 'កកួរ', 'កកើត', 'កកេបកកាប', 'កកេរ', 'កកេរឆ្អឹង', 'កកេះ', 'កកេះកកាយ', 'កកែ', 'កកែកកកោក', 'កកែកករ', 'កកែងកកោង', 'កកែងកោង', 'កកែប', 'កកែបក្របី', 'កកែបគោ', 'កកែះ', 'កកោក', 'កកោកកកាក', 'កកោស', 'កកោះ', 'កក់', 'កក់ក្ដៅ', 'កក់ក្តៅ', 'កក់ក្បាល', 'កក់ខែ', 'កក់ចែងម៉ែង', 'កក់ជ្រុង', 'កក់មូល', 'កក់លុយ', 'កក់សក់', 'កក៌ដ', 'កក្កដ', 'កក្កដា', 'កក្កដៈ', 'កក្ខលភាព', 'កក្ងិច', 'កក្ងៀក', 'កក្ងៀង', 'កក្ដដ', 'កក្តិក', 'កក្រិត', 'កក្រូស', 'កក្រើក', 'កក្រើកទឹកកក្រើកដី', 'កក្រើករំជួល', 'កក្រើករំពើក', 'កក្រោក', 'កក្រោះ', 'កក្លាក់', 'កក្លិកកក្លាក់', 'កក្វិសកក្វាស់', 'កក្សា', 'កក្អម', 'កក្អាក', 'កក្អាកកក្អាយ', 'កក្អាកក្អាយ', 'កក្អាយ', 'កក្អិច', 'កក្អិចកក្អៀក', 'កក្អិល', 'កក្អឹក', 'កក្អឹកកក្អិល', 'កក្អៀក', '

In [2]:
from khmernlp.corpus import english_words

eng_words = english_words()

print(f"Length of English words {len(eng_words)}")
print(eng_words[:100])

Length of English words 235892
['elcaja', 'problockade', 'chalkiness', 'advise', 'steeplelike', 'strumpetry', 'zoetropic', 'earthborn', 'enfetter', 'Odontophoridae', 'oilfish', 'offertorial', 'tutorless', 'metallify', 'stalklike', 'calumniousness', 'epidermatoid', 'Palamedea', 'Gradgrind', 'thyroprivic', 'interfamily', 'Jesuitess', 'aryepiglottic', 'Babbie', 'clearedness', 'revenger', 'exudative', 'Chamorro', 'herbman', 'perimysial', 'eutomous', 'westernization', 'Tarsonemidae', 'suppedaneum', 'agglutinize', 'dingy', 'culver', 'amentulum', 'carberry', 'inconsequentially', 'cribellum', 'lacunulose', 'chargee', 'certiorari', 'museology', 'unquibbling', 'morphinomania', 'phenolsulphonate', 'lithologic', 'ferromagnetic', 'Pactolus', 'shaggily', 'prethoughtfulness', 'overstraiten', 'murderously', 'nondeportation', 'dialogist', 'Aten', 'sulforicinate', 'predisaster', 'sawbones', 'follis', 'portrayment', 'electrionic', 'apocalyptist', 'foundationless', 'pukeweed', 'veined', 'sittringy', 'omni

In [3]:
from khmernlp.corpus import khmer_dict

entries = khmer_dict('កក់ក្ដៅ')
print(entries)
# Output: [{'t_main': 'ក', 't_pron': '[ក]', 't_poly': '', 't_pos': 'ន.', 't_exp': '...', 't_exam': '...'}, ...]


[{'t_main': 'កក់ក្ដៅ', 't_pron': '[កក់-ក្ដៅ]', 't_poly': '១)', 't_pos': 'កិ.', 't_exp': 'ក្ដៅល្មម ក្ដៅស្រួល :', 't_exam': 'ភួយនេះកក់ក្ដៅណាស់។'}, {'t_main': 'កក់ក្ដៅ', 't_pron': '[កក់-ក្ដៅ]', 't_poly': '២)', 't_pos': 'កិ., គុ.កិ.', 't_exp': '(ព.ប្រ.) ពេញចិត្ត រីករាយ, សង្ឃឹម ជឿជាក់ ទុកចិត្ត :', 't_exam': 'ពូកក់ក្ដៅដោយក្មួយមានការងារធ្វើ, ទទួលស្វាគមន៍យ៉ាងកក់ក្ដៅ។'}, {'t_main': 'កក់ក្ដៅ', 't_pron': '[កក់-ក្ដៅ]', 't_poly': nan, 't_pos': 'គុ.', 't_exp': 'ដែលជឿជាក់ ដែលទុកចិត្ត :', 't_exam': 'គ្រួសារកក់ក្ដៅ។'}]
